# Purpose of Our Website:
The purpose of our website is to allow students to make their school life easier, whether it's through making friends, chatting with their classmates, or knowing their schedule and when each class ends. 

# Purpose of My Feature
The purpose of my feature is a chatroom to allow classmates to discuss with each other. My program enables innovation through 

# CRUD Operations:
In building the chatroom for Neptune for Students, I implemented the four fundamental CRUD operations:

GET (Read) – Retrieves all chat messages as an array.

POST (Create) – Allows users to send new messages.

PUT (Update) – Enables users to edit their existing messages.

DELETE (Delete) – Removes a message from the chat.

Each Message in the Chatroom contains:

| Column    | Type   | Description |
|-----------|--------|-------------|
| id        | Number | A unique identifier for each message. |
| user      | String | The username of the person sending the message. |
| text      | String | The content of the message. |

# 1. Create ( POST)

The sendMessage function allows users to create new chat messages. When a user enters their name and message, it emits a chat_message event to the server. The message is then displayed in the chat.

In [ ]:
function sendMessage() {
  const username = document.getElementById("usernameInput").value;
  const message = document.getElementById("messageInput").value;

  if (username && message) {
    const messageId = Date.now(); // Generate a unique ID for the message
    socket.emit("chat_message", { id: messageId, user: username, text: message });
    document.getElementById("messageInput").value = "";
  }
}

Backend:

The backend listens for the chat_message event. When a new message is received, it creates a new Message object and stores it in the database with msg.create().After saving the message, the backend emits the chat_message event back to all connected clients (broadcast=True) to display the new message.

In [ ]:
@socketio.on('chat_message')
def handle_chat_message(data):
    msg = Message(data["text"], data["user"])
    msg.create()  # Save the message in the database
    emit('chat_message', data, broadcast=True)  # Broadcast the new message to all clients

# 2. Read (GET: Display Messages in Chat)

When the server broadcasts a new message, the chat_message event updates the UI by calling displayMessage, which adds the message to the chat.

In [ ]:
socket.on("chat_message", (data) => {
    displayMessage(data.id, `${data.user}: ${data.text}`);
  });
  

The function displayMessage creates a list item for the message and appends it to the chat list.

In [ ]:
function displayMessage(id, message) {
    const messageList = document.getElementById("messages");
    const newMessage = document.createElement("li");
    newMessage.id = id;
  
    const messageText = document.createElement("span");
    messageText.textContent = message;
  
    newMessage.appendChild(messageText);
    messageList.appendChild(newMessage);
  }  

Backend:

When a client connects, the server retrieves all stored messages from the database using Message.query.all() and emits each message using emit('chat_message', ...). It also sends a welcome message ('Welcome to the chat!') to the client.

In [ ]:
@socketio.on('connect')
def handle_connect():
    messages = Message.query.all()  # Get all messages from the database
    for message in messages:
        emit('chat_message', {"user": message._user, "text": message._content, "id": message.id})  # Send each message to the client
    emit('chat_message', {'user': 'Server', 'text': 'Welcome to the chat!'})  # Send a welcome message


# 3. Update (PUT: Edit a Message)

Users can update messages by clicking the Update button. This triggers a prompt where they can enter the new message content. The updated message is then sent to the server using the chat_update event.

In [ ]:
updateButton.onclick = () => {
    let __msg = prompt("New Message:");
    if (!__msg) return;
  
    socket.emit("chat_update", { id, content: __msg })
    messageText.textContent = `${message.split(": ")[0]}: ${__msg}`;
  };  

When the server processes the update, it sends back a chat_up event, which modifies the message in the chat.

In [ ]:
socket.on("chat_up", (data) => {
    const messEl = document.getElementById(data.id);
    messEl.textContent = data.content;
  });  

Backend:

The backend listens for the chat_update event. When it receives the event, it checks if the message exists and then updates its content in the database (msg.update({"content": new_content})). Once the message is updated, the server emits the chat_up event to all clients to reflect the changes.

In [ ]:
@socketio.on('chat_update')
def handle_chat_update(data):
    message_id = data.get("id")
    new_content = data.get("content")
    if not message_id or not new_content:
        return emit("error", {"message": "Invalid data: 'id' and 'content' are required"}, broadcast=False)

    msg = Message.query.get(message_id)
    if msg is None:
        return emit("error", {"message": "Message not found"}, broadcast=False)

    msg.update({"content": new_content})  # Update the message content in the database
    emit("chat_up", {"data": msg.read()}, broadcast=True)  # Broadcast the updated message to all clients

# 4. Delete (DELETE: Remove a Message)

Users can delete messages by clicking the Delete button, which triggers the chat_delete event. This removes the message from both the UI and the server.

In [ ]:
deleteButton.onclick = () => {
    socket.emit("chat_delete", { id });
    document.getElementById(id).remove();
  };  

When the server confirms the deletion, it sends a chat_update event, ensuring the message is removed from all clients.

In [ ]:
socket.on("chat_update", (data) => {
    const messEl = document.getElementById(data.id);
    messEl.remove();
  });  

Backend:

The backend listens for the chat_delete event. It then deletes the corresponding message from the database (msg.delete()) and then emits the chat_del event to remove the message from all clients.

In [ ]:
@socketio.on('chat_delete')
def handle_chat_delete(data):
    msg = Message.query.get(data["id"])
    msg.delete()  # Delete the message from the database
    emit('chat_del', {"id": data["id"]}, broadcast=True)  # Broadcast the deletion to all clients
